In [142]:
import pdfplumber
import pandasql as psql
import re
from prettytable import PrettyTable

def extraer_texto_pdf(ruta_pdf):
    texto_completo = ""
    
    with pdfplumber.open(ruta_pdf) as pdf:
        for page in pdf.pages:
            texto_completo += page.extract_text()
    
    return texto_completo




In [143]:
ruta_pdf = 'resumen_septiembre.pdf'
texto = extraer_texto_pdf(ruta_pdf)


In [144]:
import re
import pandas as pd

def encolumnar_datos(texto):
    """
    Esta función recibe una cadena de texto con datos en formato no estructurado y los convierte en columnas.
    
    :param texto: Cadena de texto con datos no estructurados.
    :return: DataFrame de pandas con los datos encolumnados.
    """
    # Separar cada línea del texto
    lineas = texto.strip().split('\n')
    
    # Crear una lista para almacenar las filas procesadas
    filas = []
    
    # Procesar cada línea para separar en columnas
    for linea in lineas:
        # Usar una expresión regular para capturar las columnas, con la columna Cuotas opcional
        patron = r'(\d{2}-\d{2}-\d{2})\s+([*K]?\s?[A-Z].+?)(?:\s+(\d{2}/\d{2}))?\s+(\d{6})\s+([\d.,]+)'
        coincidencia = re.match(patron, linea)
        
        if coincidencia:
            fecha, descripcion, cuotas, comprobante, monto = coincidencia.groups()
            # Si no hay información en Cuotas, dejar el campo vacío
            cuotas = cuotas if cuotas else ''
            filas.append([fecha, descripcion.strip(), cuotas, comprobante, monto])
    
    # Crear un DataFrame con las filas procesadas
    df = pd.DataFrame(filas, columns=['Fecha', 'Descripción', 'Cuotas', 'Comprobante', 'Monto'])
    
    return df


In [145]:
#Setup de opciones para la visualizacion de los DF correctamente

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [146]:
df_sumary_card = (encolumnar_datos(texto))

In [147]:
print(encolumnar_datos(texto))

       Fecha                            Descripción Cuotas Comprobante  \
0   23-06-24                    K MERPAGO*TCTIENDAS  03/06      222697   
1   04-07-24                          K MERPAGO*PSA  03/06      212596   
2   09-08-24                 K MERPAGO*MERCADOLIBRE  02/06      143241   
3   22-08-24         * MEDICUS SA DEB A 12688472000             545063   
4   23-08-24                     K MERPAGO*SPORTCLU             474832   
5   23-08-24                                * RAPPI             004033   
6   24-08-24   P MICROSOFT*1 MES Z62JNQ1XVP8B115776             074008   
7   24-08-24                * QUOTIDIANO-QUOTIDIANO             838912   
8   24-08-24                     * KUKE SRL - AXION             001656   
9   24-08-24                                * FIERA             000385   
10  25-08-24                   K MERPAGO*GOOPYSTORE             148977   
11  26-08-24  V LinkedIn Pre P77 LinkedIn Pre P7767             857802   
12  26-08-24                          

In [148]:
# Funcion para identificar cantidad de consumos en un mismo local:

# Contar la frecuencia de cada valor en la columna 'Descripción'
conteo = df_sumary_card['Descripción'].value_counts()

# Mapear el conteo de frecuencias a la columna del DataFrame
df_sumary_card['q_repetido'] = df_sumary_card['Descripción'].map(conteo)

#------------------------------------------------------------------------------------
# Funcion para identificar consumos duplicados:

df_sumary_card['concat_desc_monto'] = df_sumary_card['Fecha'] + '' +df_sumary_card['Descripción'] + ' ' + df_sumary_card['Monto']

# Contar la frecuencia de cada valor en la columna 'Descripción'
conteo_consum_dupli = df_sumary_card['concat_desc_monto'].value_counts()

# Mapear el conteo de frecuencias a la columna del DataFrame
df_sumary_card['consumo_duplicado'] = df_sumary_card['concat_desc_monto'].map(conteo_consum_dupli) 

In [149]:
# Construccion de categorias:

categorias = {
    '* TIENDA DE CAFE': 'salida',
    '* RAPPI ARG S.A.S.': 'comida',
    '* MOLINA PANADERIA': 'salida',
    '* DIA TIENDA 409': 'comida',
    '* FARMACITY': 'medicina',
    'P MICROSOFT*1 MES Z62PJZACGU5Q115776': 'suscripcion',
    '* MEDICUS SA DEB A 12688472000': 'obra social',
    '* RES BOYACA': 'comida',
    'V LinkedIn Pre P38 LinkedIn Pre P3821': 'suscripcion',
    '* CARREFOUR EXPRESS AV CARA': 'comida',
    '* PERSONAL FLOW 100329902571001': 'servicios',
    'K HAVANNA SA': 'salidas',
    '* ALMACEN DE PIZZAS': 'salidas',
    '* EDESUR DEB AUT 056340211': 'servicios',
    '* MULTIPASTA': 'comida',
    '* RAPPI': 'comida',
    'K MERPAGO*SPORTCLU': 'gimansio',
    'P MICROSOFT*1 MES Z62FLR7HJIKF115776': 'suscripcion',
    '* LA TROPILLA CARNICERIA ': 'comida',
    '* CAFE REGISTRADO': 'salida',
    '* OTAWA DELI S.A.': 'comida',
    '* DLO*RAPPI.': 'comida',
    '* RUIZ LA PASTELERIA': 'comida',
    '* FOGOSA': 'salida',
    '* LA BUENOS AIRES PASTEL': 'salida',
    
}

df_sumary_card['categoria'] = df_sumary_card['Descripción'].map(categorias)

In [150]:
# Frmato visual para la tabla de salda

x = PrettyTable()
x.field_names = df_sumary_card.columns.tolist()
for row in df_sumary_card.itertuples(index=False):
    x.add_row(row)



In [151]:
print(x)

+----------+---------------------------------------+--------+-------------+------------+------------+--------------------------------------------------------+-------------------+-------------+
|  Fecha   |              Descripción              | Cuotas | Comprobante |   Monto    | q_repetido |                   concat_desc_monto                    | consumo_duplicado |  categoria  |
+----------+---------------------------------------+--------+-------------+------------+------------+--------------------------------------------------------+-------------------+-------------+
| 23-06-24 |          K MERPAGO*TCTIENDAS          | 03/06  |    222697   | 37.539,66  |     1      |         23-06-24K MERPAGO*TCTIENDAS 37.539,66          |         1         |     nan     |
| 04-07-24 |             K MERPAGO*PSA             | 03/06  |    212596   | 47.300,00  |     1      |            04-07-24K MERPAGO*PSA 47.300,00             |         1         |     nan     |
| 09-08-24 |         K MERPAGO*MERC